In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import html5lib

path = 'example.html'
with open(path, 'rt') as myfile:
    data = myfile.read().replace("<br>", '\n')


df = pd.read_html(data)

In [2]:
# Since the older version cape uses a much simpler HTML tree, this is wayy easier to clean
pd.read_html("07capessample.html")[0]

,0,1,2,3,4,5
0,VIS,VIS 60,(A),"Michaelis, Danielle Ruth","Michaelis, Danielle Ruth",SU07
1,NaN,NaN,NaN,Enrollment: 19,Questionnaire's Returned: 18,SelfCAPE


In [3]:
df[3]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,PLEASE COMMENT ON THE FOLLOWING:,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,NaN,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,Custom Question 5,0,0.00,NaN


In [4]:
soup = BeautifulSoup(open("example.html", "r"))

In [5]:
import re

In [6]:
# seperate question, column head, response for each question using dictionaries --> then create dataframe to fit them

In [78]:
def question_collector(bs4obj):
    question_text = bs4obj.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_.*")})
    dct_key = []
    return_dct = {}
    counter = 0
    for qt in question_text:
        if qt.text.split()[0].isupper() and len(qt.text.split()[0]) > 1:
            dct_key.append(qt.text)
            return_dct[qt.text] = {}
            counter += 1
        else:
            if len(soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_" 
                                                                + str(counter)+"_.*")})) != 0:
                col = dict(zip([i.text for i in soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_" 
                                                                    + str(counter)+"_.*")})],
                                [[i.get_text(separator=" ").split(" ")[0]] for i in soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoices_" 
                                                                    + str(counter)+"_.*")})]))
                col_copy = col.copy()
                col_copy.update({"Question": qt.text})
                return_dct[dct_key[-1]].update({qt.text: col_copy})
                counter += 1
            else:
                col = dict(zip(col.keys(), [[i.get_text(separator=" ").split(" ")[0]] for i in soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoices_" 
                                                                    + str(counter)+"_.*")})]))
                col_copy = col.copy()
                col_copy.update({"Question": qt.text})
                return_dct[dct_key[-1]].update({qt.text: col_copy})
                counter += 1
    return return_dct

In [89]:
question_collector(soup)

{'PLEASE COMMENT ON THE FOLLOWING:': {'Your class level is': {'Freshman': ['10'],
   'Sophomore': ['16'],
   'Junior': ['7'],
   'Senior': ['7'],
   'Graduate': ['0'],
   'Extension': ['0'],
   'Visitor': ['0'],
   'Question': 'Your class level is'},
  'Your reason for taking this class is': {'Major': ['1'],
   'Minor': ['1'],
   'Gen. Ed.': ['29'],
   'Elective': ['6'],
   'Interest': ['3'],
   'Question': 'Your reason for taking this class is'},
  'What grade do you expect in this class?': {'A': ['26'],
   'B': ['8'],
   'C': ['2'],
   'D': ['0'],
   'F': ['0'],
   'P': ['4'],
   'NP': ['0'],
   'Question': 'What grade do you expect in this class?'}},
 'GENERAL QUESTIONS': {'I learned a great deal from this course.': {'Strongly Disagree': ['1'],
   'Disagree': ['2'],
   'Neither Agree nor Disagree': ['3'],
   'Agree': ['16'],
   'Strongly Agree': ['18'],
   'Not Applicable': ['0'],
   'Question': 'I learned a great deal from this course.'},
  'How many hours a week do you spend study

In [88]:
example_query = question_collector(soup)['PLEASE COMMENT ON THE FOLLOWING:']['Your class level is']
example_query

{'Freshman': ['10'],
 'Sophomore': ['16'],
 'Junior': ['7'],
 'Senior': ['7'],
 'Graduate': ['0'],
 'Extension': ['0'],
 'Visitor': ['0'],
 'Question': 'Your class level is'}

In [90]:
{"Test": pd.DataFrame(data = example_query).set_index("Question")}

{'Test':                     Freshman Sophomore Junior Senior Graduate Extension  \
 Question                                                                  
 Your class level is       10        16      7      7        0         0   
 
                     Visitor  
 Question                     
 Your class level is       0  }

In [ ]:
def question_collector_to_pd(bs4obj):
    question_text = bs4obj.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_.*")})
    dct_key = []
    return_dct = {}
    counter = 0
    for qt in question_text:
        if qt.text.split()[0].isupper() and len(qt.text.split()[0]) > 1:
            dct_key.append(qt.text)
            return_dct[qt.text] = {}
            counter += 1
        else:
            if len(soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_" 
                                                                + str(counter)+"_.*")})) != 0:
                col = dict(zip([i.text for i in soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_" 
                                                                    + str(counter)+"_.*")})],
                                [[i.get_text(separator=" ").split(" ")[0]] for i in soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoices_" 
                                                                    + str(counter)+"_.*")})]))
                col_copy = col.copy()
                col_copy.update({"Question": qt.text})
                return_dct[dct_key[-1]].update({qt.text: col_copy})
                counter += 1
            else:
                col = dict(zip(col.keys(), [[i.get_text(separator=" ").split(" ")[0]] for i in soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoices_" 
                                                                    + str(counter)+"_.*")})]))
                col_copy = col.copy()
                col_copy.update({"Question": qt.text})
                return_dct[dct_key[-1]].update({qt.text: col_copy})
                counter += 1
    return return_dct

In [33]:
soup.find_all("span", {"id": 
                       re.compile("ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_.*")})

[<span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_0">10<br/>25%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_1">16<br/>40%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_2">7<br/>18%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_3">7<br/>18%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_4">0<br/>0%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_5">0<br/>0%<br/></span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_rptChoices_1_rbSelect_6">0<br/>0%<br/></span>]

In [30]:
re.search("ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_", "ContentPlaceHolder1_dlQuestionnaire_rptChoiceTexts_1_lblChoiceText_0")

<re.Match object; span=(0, 51), match='ContentPlaceHolder1_dlQuestionnaire_rptChoiceText>

In [14]:
soup.find_all("span", {"id": re.compile("ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_.*")})

[<span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_0">PLEASE COMMENT ON THE FOLLOWING:</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_1">Your class level is</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_2">Your reason for taking this class is</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_3">What grade do you expect in this class?</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_4">GENERAL QUESTIONS</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_5">I learned a great deal from this course.</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_6">How many hours a week do you spend studying outside of class on average?</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_7">How often do you attend this course?</span>,
 <span id="ContentPlaceHolder1_dlQuestionnaire_lblQuestionText_8">COURSE MATERIAL <b>AAS 10</b></span>,
 <spa